In [1]:
import os
import pandas as pd 
from mlproject import logger

[2025-05-10 23:07:41,396 : INFO : __init__ : Logger has been set up successfully!]


In [2]:
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction\\exp'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str
    label_encoder: Path
    preprocessor_path: Path
    columns_to_drop: list
    num_cols: list
    cat_cols: list

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column=config.target_column,
            preprocessor_path=config.preprocessor_path,
            label_encoder=config.label_encoder,
            columns_to_drop=schema.columns_to_drop,
            num_cols=schema.num_cols,
            cat_cols=schema.cat_cols
        )
        return data_transformation_config

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
import joblib
from mlproject import logger

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.label_encoders = {}
        self.num_cols = config.num_cols
        self.cat_cols_le =  config.cat_cols
        self.cols_to_drop = config.columns_to_drop

    def preprocess_data(self, data: pd.DataFrame) -> pd.DataFrame:
    
            data = data.copy()
            data = data.drop(columns=self.cols_to_drop, errors='ignore')
            
            for col in self.num_cols:
                if col in data.columns:
                    data[col] = pd.to_numeric(data[col], errors='coerce')
                    data[col] = data[col].fillna(data[col].median())
            
            # Label encode categorical columns
            for column in self.cat_cols_le:
                if column in data.columns:
                    le = LabelEncoder()
                    data[column] = le.fit_transform(data[column].astype(str))
                    self.label_encoders[column] = le
            
            # Encode target column if categorical
            if self.config.target_column in data.columns:
                le = LabelEncoder()
                data[self.config.target_column] = le.fit_transform(data[self.config.target_column])
                self.label_encoders[self.config.target_column] = le
            
            # Save label encoders
            os.makedirs(os.path.dirname(self.config.label_encoder), exist_ok=True)
            joblib.dump(self.label_encoders, self.config.label_encoder)
            
            return data
            

    def train_test_spliting(self) -> tuple:
            data = pd.read_csv(self.config.data_path)
            data = self.preprocess_data(data)
            
            X = data.drop(columns=[self.config.target_column])
            y = data[self.config.target_column]
            
            smote = SMOTE(random_state=42)
            X_resampled, y_resampled = smote.fit_resample(X, y)
            
            X_resampled = pd.DataFrame(X_resampled, columns=X.columns)
            resampled_data = X_resampled.copy()
            resampled_data[self.config.target_column] = y_resampled
            
            train, test = train_test_split(resampled_data, test_size=0.25, random_state=42)
            
            train_path = os.path.join(self.config.root_dir, "train.csv")
            test_path = os.path.join(self.config.root_dir, "test.csv")
            train.to_csv(train_path, index=False)
            test.to_csv(test_path, index=False)
            
            logger.info(f"Original data shape: {data.shape}")
            logger.info(f"Resampled data shape: {resampled_data.shape}")
            logger.info(f"Training data shape: {train.shape}")
            logger.info(f"Test data shape: {test.shape}")
            
            return train, test
            
    def preprocess_features(self, train, test) -> tuple:

        logger.info(f"Numeric columns for transformation: {self.num_cols}")
            
        valid_num_cols = [col for col in self.num_cols if col in train.columns]

        if not valid_num_cols:
            raise ValueError(f"None of the specified numeric columns {self.num_cols} exist in the data")
            
        numeric_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])
            
        preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, valid_num_cols)
                ],
                remainder='passthrough'
        )

        # Split features and target
        train_x = train.drop(columns=[self.config.target_column])
        test_x = test.drop(columns=[self.config.target_column])
        train_y = train[self.config.target_column]
        test_y = test[self.config.target_column]

        train_processed = preprocessor.fit_transform(train_x)
        test_processed = preprocessor.transform(test_x)

        # Reshape 
        train_y = train_y.values.reshape(-1, 1)
        test_y = test_y.values.reshape(-1, 1)

        train_combined = np.hstack((train_processed, train_y))
        test_combined = np.hstack((test_processed, test_y))

        os.makedirs(os.path.dirname(self.config.preprocessor_path), exist_ok=True)
        joblib.dump(preprocessor, self.config.preprocessor_path)
            
        np.save(os.path.join(self.config.root_dir, "train_processed.npy"), train_combined)
        np.save(os.path.join(self.config.root_dir, "test_processed.npy"), test_combined)

        logger.info(f"Preprocessor saved at: {self.config.preprocessor_path}")
        logger.info(f"Training data shape: {train_processed.shape}")
        logger.info(f"Testing data shape: {test_processed.shape}")
            
        return train_processed, test_processed
            

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    train,test = data_transformation.train_test_spliting()
    train_processed, test_processed = data_transformation.preprocess_features(train, test)

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-05-10 23:07:42,758 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-05-10 23:07:42,762 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-05-10 23:07:42,769 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-05-10 23:07:42,772 : INFO : common : created directory at: artifacts]
[2025-05-10 23:07:42,776 : INFO : common : created directory at: artifacts/data_transformation]
[2025-05-10 23:07:43,050 : INFO : 4258982600 : Original data shape: (7043, 20)]
[2025-05-10 23:07:43,051 : INFO : 4258982600 : Resampled data shape: (10348, 20)]
[2025-05-10 23:07:43,052 : INFO : 4258982600 : Training data shape: (7761, 20)]
[2025-05-10 23:07:43,053 : INFO : 4258982600 : Test data shape: (2587, 20)]
[2025-05-10 23:07:43,055 : INFO : 4258982600 : Numeric columns for transformation: ['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen']]
[2025-05-10 23:07:43,103 : INFO : 4258982600 : Preprocessor saved at: artifacts/data_transformat